In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense,
    Flatten, Dropout, BatchNormalization
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import os


##CNN

LOAD DATASET

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

BASE_DIR = "/content/drive/MyDrive/semester7/uap/Vegetable Images"

train_dir = f"{BASE_DIR}/train"
val_dir   = f"{BASE_DIR}/val"
test_dir  = f"{BASE_DIR}/test"



Image Generator

In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 5000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


MODEL CNN (FROM SCRATCH)

In [ ]:
NUM_CLASSES = 5

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])


In [ ]:
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

cnn_model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,246,853 (84.87 MB)

 Trainable params: 22,246,405 (84.86 MB)

 Non-trainable params: 448 (1.75 KB)

TRAINING & SIMPAN MODEL

In [ ]:
checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/uap/model_cnn_sayur.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


In [ ]:
history = cnn_model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7518 - loss: 10.1344
Epoch 1: val_accuracy improved from -inf to 0.24500, saving model to /content/drive/MyDrive/uap/model_cnn_sayur.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 517s 3s/step - accuracy: 0.7523 - loss: 10.1194 - val_accuracy: 0.2450 - val_loss: 57.8577
Epoch 2/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - accuracy: 0.8825 - loss: 4.2548
Epoch 2: val_accuracy improved from 0.24500 to 0.39000, saving model to /content/drive/MyDrive/uap/model_cnn_sayur.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 85s 537ms/step - accuracy: 0.8826 - loss: 4.2478 - val_accuracy: 0.3900 - val_loss: 23.7189
Epoch 3/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.9308 - loss: 1.5445
Epoch 3: val_accuracy improved from 0.39000 to 0.64500, saving model to /content/drive/MyDrive/uap/model_cnn_sayur.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 87s 552ms/step - accuracy: 0.9308 - loss: 1.5432 - val_accuracy: 0.6450 - val_loss: 9.0860
Epoch 4/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 502ms/step - accuracy: 0.9314 - loss: 1.1808
Epoch 4: val_accuracy improved from 0.64500 to 0.90800, saving model to /content/drive/MyDrive/uap/model_cnn_sayur.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 87s 557ms/step - accuracy: 0.9314 - loss: 1.1784 - val_accuracy: 0.9080 - val_loss: 1.2326
Epoch 5/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - accuracy: 0.9315 - loss: 0.8871
Epoch 5: val_accuracy improved from 0.90800 to 0.96300, saving model to /content/drive/MyDrive/uap/model_cnn_sayur.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 88s 562ms/step - accuracy: 0.9315 - loss: 0.8857 - val_accuracy: 0.9630 - val_loss: 0.1758
Epoch 6/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 505ms/step - accuracy: 0.9342 - loss: 0.3982
Epoch 6: val_accuracy did not improve from 0.96300
157/157 ━━━━━━━━━━━━━━━━━━━━ 83s 529ms/step - accuracy: 0.9342 - loss: 0.3990 - val_accuracy: 0.8630 - val_loss: 2.3288
Epoch 7/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step - accuracy: 0.9508 - loss: 0.3610
Epoch 7: val_accuracy did not improve from 0.96300
157/157 ━━━━━━━━━━━━━━━━━━━━ 77s 490ms/step - accuracy: 0.9508 - loss: 0.3615 - val_accuracy: 0.8430 - val_loss: 3.2744
Epoch 8/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.9301 - loss: 0.5112
Epoch 8: val_accuracy did not improve from 0.96300
157/157 ━━━━━━━━━━━━━━━━━━━━ 76s 486ms/step - accuracy: 0.9301 - loss: 0.5111 - val_accuracy: 0.9620 - val_loss: 0.4113
Epoch 9/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9465 - loss: 0.4335
Epoch 9: val_ac

EVALUASI MODEL CNN

In [ ]:
loss, acc = cnn_model.evaluate(test_data)
print("Test Accuracy:", acc)


32/32 ━━━━━━━━━━━━━━━━━━━━ 259s 8s/step - accuracy: 0.9672 - loss: 0.5031
Test Accuracy: 0.9620000123977661


In [ ]:
import json

class_indices = train_data.class_indices
with open("/content/drive/MyDrive/label_sayur.json", "w") as f:
    json.dump(class_indices, f)


##MOBILENETV2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import json


PATH & PARAMETER

In [ ]:
BASE_DIR = "/content/drive/MyDrive/semester7/uap/Vegetable Images"

train_dir = BASE_DIR + "/train"
val_dir   = BASE_DIR + "/val"
test_dir  = BASE_DIR + "/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5


IMAGE GENERATOR

In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen  = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 5000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


MODEL MOBILENETV2

In [ ]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Classification Head

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

mobilenet_model = Model(inputs=base_model.input, outputs=output)


Compile Model

In [ ]:
mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

mobilenet_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,587,205 (9.87 MB)

 Trainable params: 329,221 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

TRAIN & SIMPAN MODEL

In [ ]:
checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/semester7/uap/model_mobilenetv2_5class.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


In [ ]:
history_mobilenet = mobilenet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 516ms/step - accuracy: 0.6829 - loss: 0.8633
Epoch 1: val_accuracy improved from -inf to 0.99400, saving model to /content/drive/MyDrive/semester7/uap/model_mobilenetv2_5class.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 108s 597ms/step - accuracy: 0.6840 - loss: 0.8607 - val_accuracy: 0.9940 - val_loss: 0.0521
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - accuracy: 0.9747 - loss: 0.0976
Epoch 2: val_accuracy improved from 0.99400 to 0.99500, saving model to /content/drive/MyDrive/semester7/uap/model_mobilenetv2_5class.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 78s 497ms/step - accuracy: 0.9747 - loss: 0.0975 - val_accuracy: 0.9950 - val_loss: 0.0239
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.9911 - loss: 0.0463
Epoch 3: val_accuracy improved from 0.99500 to 0.99700, saving model to /content/drive/MyDrive/semester7/uap/model_mobilenetv2_5class.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 77s 494ms/step - accuracy: 0.9911 - loss: 0.0462 - val_accuracy: 0.9970 - val_loss: 0.0131
Epoch 4/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step - accuracy: 0.9941 - loss: 0.0312
Epoch 4: val_accuracy improved from 0.99700 to 0.99800, saving model to /content/drive/MyDrive/semester7/uap/model_mobilenetv2_5class.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 76s 484ms/step - accuracy: 0.9941 - loss: 0.0312 - val_accuracy: 0.9980 - val_loss: 0.0096
Epoch 5/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step - accuracy: 0.9962 - loss: 0.0216
Epoch 5: val_accuracy improved from 0.99800 to 0.99900, saving model to /content/drive/MyDrive/semester7/uap/model_mobilenetv2_5class.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 75s 479ms/step - accuracy: 0.9962 - loss: 0.0216 - val_accuracy: 0.9990 - val_loss: 0.0070
Epoch 6/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - accuracy: 0.9965 - loss: 0.0172
Epoch 6: val_accuracy did not improve from 0.99900
157/157 ━━━━━━━━━━━━━━━━━━━━ 76s 479ms/step - accuracy: 0.9965 - loss: 0.0172 - val_accuracy: 0.9980 - val_loss: 0.0077
Epoch 7/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.9970 - loss: 0.0151
Epoch 7: val_accuracy did not improve from 0.99900
157/157 ━━━━━━━━━━━━━━━━━━━━ 75s 479ms/step - accuracy: 0.9970 - loss: 0.0151 - val_accuracy: 0.9980 - val_loss: 0.0083
Epoch 8/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step - accuracy: 0.9964 - loss: 0.0111
Epoch 8: val_accuracy did not improve from 0.99900
157/157 ━━━━━━━━━━━━━━━━━━━━ 74s 471ms/step - accuracy: 0.9964 - loss: 0.0111 - val_accuracy: 0.9990 - val_loss: 0.0041
Epoch 9/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step - accuracy: 0.9979 - loss: 0.0096
Epoch 9: val_ac

157/157 ━━━━━━━━━━━━━━━━━━━━ 76s 485ms/step - accuracy: 0.9990 - loss: 0.0040 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 20/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step - accuracy: 0.9995 - loss: 0.0034
Epoch 20: val_accuracy did not improve from 1.00000
157/157 ━━━━━━━━━━━━━━━━━━━━ 75s 477ms/step - accuracy: 0.9995 - loss: 0.0034 - val_accuracy: 0.9990 - val_loss: 0.0023


In [ ]:
loss, acc = mobilenet_model.evaluate(test_data)
print("Test Accuracy MobileNetV2:", acc)


32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 134ms/step - accuracy: 0.9998 - loss: 7.7830e-04
Test Accuracy MobileNetV2: 0.9990000128746033


##VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import json


PATH & PARAMETER

In [ ]:
BASE_DIR = "/content/drive/MyDrive/semester7/uap/Vegetable Images"

train_dir = BASE_DIR + "/train"
val_dir   = BASE_DIR + "/val"
test_dir  = BASE_DIR + "/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5


IMAGE GENERATOR

In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
val_gen  = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)
val_data = val_gen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)
test_data = test_gen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)


Found 5000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


MODEL VGG16 (FREEZE BASE)

In [ ]:
base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Head Klasifikasi

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

vgg_model = Model(inputs=base_model.input, outputs=output)


Compile

In [ ]:
vgg_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
vgg_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,301 (56.64 MB)

 Trainable params: 132,613 (518.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

TRAIN & SIMPAN MODEL

In [ ]:
checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/semester7/uap/model_vgg16_5class.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history_vgg = vgg_model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 523ms/step - accuracy: 0.9849 - loss: 0.0647
Epoch 1: val_accuracy improved from -inf to 0.99900, saving model to /content/drive/MyDrive/semester7/uap/model_vgg16_5class.h5


157/157 ━━━━━━━━━━━━━━━━━━━━ 90s 572ms/step - accuracy: 0.9849 - loss: 0.0646 - val_accuracy: 0.9990 - val_loss: 0.0200
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 520ms/step - accuracy: 0.9815 - loss: 0.0651
Epoch 2: val_accuracy did not improve from 0.99900
157/157 ━━━━━━━━━━━━━━━━━━━━ 87s 555ms/step - accuracy: 0.9815 - loss: 0.0650 - val_accuracy: 0.9990 - val_loss: 0.0194
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 518ms/step - accuracy: 0.9874 - loss: 0.0583
Epoch 3: val_accuracy did not improve from 0.99900
157/157 ━━━━━━━━━━━━━━━━━━━━ 87s 554ms/step - accuracy: 0.9874 - loss: 0.0583 - val_accuracy: 0.9990 - val_loss: 0.0187
Epoch 4/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 518ms/step - accuracy: 0.9854 - loss: 0.0542
Epoch 4: val_accuracy did not improve from 0.99900
157/157 ━━━━━━━━━━━━━━━━━━━━ 87s 555ms/step - accuracy: 0.9854 - loss: 0.0542 - val_accuracy: 0.9990 - val_loss: 0.0175
Epoch 5/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 520ms/step - accuracy: 0.9889 - loss: 0.0514
Epoch 5: val_ac

In [ ]:
loss, acc = vgg_model.evaluate(test_data)
print("Test Accuracy VGG16:", acc)


32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 161ms/step - accuracy: 0.9954 - loss: 0.0204
Test Accuracy VGG16: 0.9959999918937683
